In [1]:
import pandas as pd

data_path = r'C:\Users\asusz\OneDrive\Робочий стіл\СА\ThyroCare\data_new.csv'

df = pd.read_csv(data_path)

In [2]:
df.drop(['Thyroid_Cancer_Risk'], axis=1, inplace = True)

In [3]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
label_enc_country = LabelEncoder()
label_enc_ethnicity = LabelEncoder()
df_encoded['Country'] = label_enc_country.fit_transform(df_encoded['Country'])
df_encoded['Ethnicity'] = label_enc_ethnicity.fit_transform(df_encoded['Ethnicity'])
label_encoder = LabelEncoder()
#df['Gender'] = label_encoder.fit_transform(df['Gender'])

df_encoded.head()

,Age,Gender,Country,Ethnicity,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Thyroid_Cancer_Risk,Diagnosis
0,66,0,6,2,0,1,0,0,0,0,9.37,1.67,6.16,1.08,1,0
1,29,0,2,3,0,1,0,0,0,0,1.83,1.73,10.54,4.05,1,0
2,86,0,5,2,0,0,0,0,0,0,6.26,2.59,10.57,4.61,1,0
3,75,1,3,1,0,0,0,0,0,0,4.10,2.62,11.04,2.46,2,0
4,35,1,2,0,1,1,0,0,0,0,9.10,2.11,10.71,2.11,3,0


In [5]:
df_encoded['Combination'] = df_encoded['Smoking'].astype(str) + df_encoded['Obesity'].astype(str) + df_encoded['Diabetes'].astype(str)
df_encoded['Combination'] = df_encoded['Combination'].apply(lambda x: int(x, 2))
print(df_encoded[['Smoking', 'Obesity', 'Diabetes', 'Combination']].head())

   Smoking  Obesity  Diabetes  Combination
0        0        0         0            0
1        0        0         0            0
2        0        0         0            0
3        0        0         0            0
4        0        0         0            0


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.utils import resample

major1 = df_encoded[df_encoded['Thyroid_Cancer_Risk'] == 1]
major2 = df_encoded[df_encoded['Thyroid_Cancer_Risk'] == 2]
minor = df_encoded[df_encoded['Thyroid_Cancer_Risk'] == 3]

major1_downsampled = resample(
    major1, 
    replace=False,     # без повторів
    n_samples=len(minor), 
    random_state=42
)

major2_downsampled = resample(
    major2, 
    replace=False,     # без повторів
    n_samples=len(minor), 
    random_state=42
)

df_balanced = pd.concat([major1_downsampled, major2_downsampled, minor])
print(df_balanced['Thyroid_Cancer_Risk'].value_counts())


Thyroid_Cancer_Risk
1    31903
2    31903
3    31903
Name: count, dtype: int64


In [13]:
df_balanced['Nodule_Size'].value_counts()

Nodule_Size
3.05    241
3.71    235
3.47    231
1.85    221
1.21    221
       ... 
0.68    161
3.20    161
2.33    154
0.00     89
5.00     78
Name: count, Length: 501, dtype: int64

In [17]:
df_balanced[df_balanced['Nodule_Size'] == 0].head(80)


,Age,Gender,Country,Ethnicity,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Thyroid_Cancer_Risk,Diagnosis,Combination
123723,17,0,2,2,0,0,0,0,0,0,8.37,2.70,7.33,0.0,1,0,0
37232,87,1,3,4,1,0,0,0,0,0,8.68,2.08,10.43,0.0,1,1,0
83603,59,1,1,0,0,0,0,0,1,1,2.90,2.89,9.36,0.0,1,0,3
67669,27,0,7,3,0,0,0,0,1,0,2.80,2.76,5.96,0.0,1,0,2
90906,31,0,5,2,0,0,0,0,0,0,2.60,2.58,6.15,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128298,56,0,3,1,0,0,1,0,0,0,0.50,1.09,9.64,0.0,3,1,0
129172,69,0,3,1,0,0,1,0,0,1,9.52,2.18,5.92,0.0,3,1,1
138154,61,0,1,1,1,0,0,0,0,0,2.55,0.66,10.22,0.0,3,1,0
143213,71,1,5,1,1,0,0,0,0,0,8.07,1.02,11.98,0.0,3,1,0


In [27]:
X = df_balanced.drop(columns=['Thyroid_Cancer_Risk','Diagnosis','Smoking', 'Obesity', 'Diabetes'])
y = df_balanced["Diagnosis"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
cols_to_scale = ['TSH_Level', 'T3_Level', 'T4_Level', 'Nodule_Size']
X_train[cols_to_scale] = scaler.fit_transform(X_train[cols_to_scale])
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features=0.5,
    random_state=42
)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.7974088392017553

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85     12729
           1       0.70      0.69      0.70      6413

    accuracy                           0.80     19142
   macro avg       0.77      0.77      0.77     19142
weighted avg       0.80      0.80      0.80     19142


Confusion Matrix:
[[10812  1917]
 [ 1961  4452]]


In [43]:
y_prob = rf_model.predict_proba(X_test)[:, 1]

threshold = 0.4
y_pred_new = (y_prob >= threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred_new)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_new))

Accuracy: 0.80

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85     12729
           1       0.70      0.70      0.70      6413

    accuracy                           0.80     19142
   macro avg       0.77      0.77      0.77     19142
weighted avg       0.80      0.80      0.80     19142



In [45]:
import joblib
import numpy as np

joblib.dump(rf_model, 'model_diag.pkl')
#np.save('threshold_diag.npy', np.array(threshold))


['model_diag.pkl']